# 課題5：映画レビューの評判分析

本課題ではAmazon傘下の「IMDb」に投稿された映画のレビュー（英語）を分析し、レビューがPositive（ポジティブ）か、Negative（ネガティブ）かの判別を行ないます。

データセットは、以下のサイトで配布されているものを利用します。

[Large Movie Review Dataset](https://ai.stanford.edu/%7Eamaas/data/sentiment/)

わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なライブラリのimport

In [ ]:
# （変更しないでください）

# 必要なライブラリのimport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 文章ファイル検索用
import glob
import collections
from sklearn.feature_extraction import DictVectorizer

# DataFrameですべての列を表示する設定
pd.options.display.max_columns = None

# seabornによる装飾を適用する
sns.set_theme()

In [117]:
# ===== 日本語フォント設定（Mac向け／文字化け対策）=====
from matplotlib import font_manager, rcParams
_jp_candidates = ["Hiragino Sans", "Hiragino Kaku Gothic ProN", "Hiragino Maru Gothic ProN", "IPAexGothic", "Noto Sans CJK JP"]
_available = {f.name for f in font_manager.fontManager.ttflist}
for _f in _jp_candidates:
    if _f in _available:
        rcParams["font.family"] = _f
        break
rcParams["axes.unicode_minus"] = False

# ======== 乱数シード設定（要件：random_state を外す/可変化）========
# None にすると固定しません。random_stateを固定化するときは 0とか42とか などの整数を入れる
# RANDOM_STATE = None  # デバッグ時はコメントアウトする
RANDOM_STATE = 42 # デバッグ時指定用
print("設定したRANDOM_STATE=",RANDOM_STATE)

設定したRANDOM_STATE= 42


## 2. データの読み込み

In [ ]:
%%capture 
# ↑ログが５万行でるので、出力を抑止
# ダウンロードした圧縮ファイルを解凍する（変更しないでください）
!tar zxvf aclImdb_v1.tar.gz

*./aclImdb* フォルダ内にあるファイルを読み込みます。

In [ ]:
# trainフォルダのファイル一覧を取得（変更しないでください）
train_neg_files = glob.glob("./aclImdb/train/neg/*")
train_pos_files = glob.glob("./aclImdb/train/pos/*")

# testフォルダのファイル一覧を取得（変更しないでください）
test_neg_files = glob.glob("./aclImdb/test/neg/*")
test_pos_files = glob.glob("./aclImdb/test/pos/*")

In [ ]:
# 中身を確認
print("train_neg_files[3]=" , train_neg_files[3] )

print("train_neg_files (first 5)=" , train_neg_files[:5])
print("train_pos_files (first 5)=" ,train_pos_files[:5])
print("test_neg_files (first 5)=" , test_neg_files[:5])
print("test_pos_files (first 5)=" , test_pos_files[:5])

train_neg_files[3]= ./aclImdb/train/neg/9056_1.txt
train_neg_files (first 5)= ['./aclImdb/train/neg/1821_4.txt', './aclImdb/train/neg/10402_1.txt', './aclImdb/train/neg/1062_4.txt', './aclImdb/train/neg/9056_1.txt', './aclImdb/train/neg/5392_3.txt']
train_pos_files (first 5)= ['./aclImdb/train/pos/4715_9.txt', './aclImdb/train/pos/12390_8.txt', './aclImdb/train/pos/8329_7.txt', './aclImdb/train/pos/9063_8.txt', './aclImdb/train/pos/3092_10.txt']
test_neg_files (first 5)= ['./aclImdb/test/neg/1821_4.txt', './aclImdb/test/neg/9487_1.txt', './aclImdb/test/neg/4604_4.txt', './aclImdb/test/neg/2828_2.txt', './aclImdb/test/neg/10890_1.txt']
test_pos_files (first 5)= ['./aclImdb/test/pos/4715_9.txt', './aclImdb/test/pos/1930_9.txt', './aclImdb/test/pos/3205_9.txt', './aclImdb/test/pos/10186_10.txt', './aclImdb/test/pos/147_10.txt']


In [ ]:
# それぞれのファイル数を確認
print("train neg:", len(train_neg_files))
print("train pos:", len(train_pos_files))
print("test  neg:", len(test_neg_files))
print("test  pos:", len(test_pos_files))

train neg: 12500
train pos: 12500
test  neg: 12500
test  pos: 12500


前処理をするため、合計50000あるファイルをリストにまとめます。

In [ ]:
# ファイル名をまとめたリストを用意（変更しないでください）
filenames = train_neg_files + train_pos_files + test_neg_files + test_pos_files

# filenamesの長さを確認（変更しないでください）
len(filenames)

50000

In [ ]:
# filenamesの中身を確認
print("filenames[0]=" , filenames[0])
print("filenames[-1]=" , filenames[-1])

filenames[0]= ./aclImdb/train/neg/1821_4.txt
filenames[-1]= ./aclImdb/test/pos/1917_10.txt


リストの最初と最後のファイルを確認してみます。

In [ ]:
# エンコーディング用定数（変更しないでください）
ENCODING = 'utf-8'

In [ ]:
# 最初のファイルの内容を確認
with open(filenames[0], "r", encoding=ENCODING, errors="ignore") as f:
    first_text = f.read()

print("len(first_text)=" ,len(first_text))
print("first_text=",first_text)

len(first_text)= 250
first_text= Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.


In [ ]:
# 最後のファイルの内容を確認
with open(filenames[-1], "r", encoding=ENCODING, errors="ignore") as f:
    last_text = f.read()

print("len(last_text)=" ,len(last_text))
print("last_text=",last_text)

len(last_text)= 553
last_text= I saw this movie on TV and loved it! I am a real disaster film fan, and this one was great. The cast was made of some really interesting people. Connie Selleca is always great. And William Devane is in a league of his own. He can play both comedy and thriller in the same movie like few others can. The story line is great too. The thought of being able to follow a time line of what will happen, and to use this time line to prevent a global disaster is an interesting idea. And this movie brings it out in such a way that is almost totally believable.


## 3. データの前処理

データの前処理として、形態素解析と行列への変換を行ないます。

### 形態素解析

In [ ]:
# 文字列の中で使われている単語ごとの数を返す関数を作成
#（レッスン本編の内容を確認して、下記にコードを追記してください）
# 文字列のなかで使われている単語ごとの数を返す
def get_word_count(text, min_length=3):
    # ノイズの除去：不要と思われる文字を除去する
    for ch in ".,:;!?-+*/=()[]{}<>~^#$@%&'\"_0123456789":
        text = text.replace(ch, ' ')

    # 形態素解析：文章を単語に分割
    _words = text.strip().split()

    # 表記のゆれの補正：
    # 単語のリストを受け取り、指定された文字数以上の単語だけをすべて小文字にして返す
    _words = [_word.lower() for _word in _words if len(_word) >= min_length]

    # collections.Counterの戻り値は辞書型のサブクラス
    _count = collections.Counter(_words)

    # 辞書型に変換して返す
    return dict(_count)

In [ ]:
# 最初のファイルを使って、先ほど作成した関数をテスト
print(list(get_word_count(first_text).items())[:10])

[('working', 1), ('with', 1), ('one', 1), ('the', 2), ('best', 1), ('shakespeare', 1), ('sources', 1), ('this', 1), ('film', 2), ('manages', 1)]


In [ ]:
# 単語ごとの数のリストを作成（変更しないでください）
word_count_data = []

In [ ]:
# すべてのファイルに対して、先ほど作成した関数を実行
# --- 再実行時の重複を防止 ---
word_count_data.clear()

for i, fn in enumerate(filenames, start=1):
    with open(fn, "r", encoding=ENCODING, errors="ignore") as f:
        text = f.read()
        counts = get_word_count(text)
        word_count_data.append(counts)

    # 進捗表示（重ければコメントアウト可）
    if i % 5000 == 0:
        print(f"processed: {i}/{len(filenames)}")

processed: 5000/50000
processed: 10000/50000
processed: 15000/50000
processed: 20000/50000
processed: 25000/50000
processed: 30000/50000
processed: 35000/50000
processed: 40000/50000
processed: 45000/50000
processed: 50000/50000


In [ ]:
# 単語ごとの数のリストの長さを確認
print("len(word_count_data):", len(word_count_data))

len(word_count_data): 50000


In [ ]:
# 中身を確認
print("word_count_data[0]", word_count_data[0])

word_count_data[0] {'working': 1, 'with': 1, 'one': 1, 'the': 2, 'best': 1, 'shakespeare': 1, 'sources': 1, 'this': 1, 'film': 2, 'manages': 1, 'creditable': 1, 'source': 1, 'whilst': 1, 'still': 1, 'appealing': 1, 'wider': 1, 'audience': 1, 'branagh': 1, 'steals': 1, 'from': 1, 'under': 1, 'fishburne': 1, 'nose': 1, 'and': 1, 'there': 1, 'talented': 1, 'cast': 1, 'good': 1, 'form': 1}


In [ ]:
# 単語ごとの数のリストの0番目を表示
word_count_data[0]

{'working': 1,
 'with': 1,
 'one': 1,
 'the': 2,
 'best': 1,
 'shakespeare': 1,
 'sources': 1,
 'this': 1,
 'film': 2,
 'manages': 1,
 'creditable': 1,
 'source': 1,
 'whilst': 1,
 'still': 1,
 'appealing': 1,
 'wider': 1,
 'audience': 1,
 'branagh': 1,
 'steals': 1,
 'from': 1,
 'under': 1,
 'fishburne': 1,
 'nose': 1,
 'and': 1,
 'there': 1,
 'talented': 1,
 'cast': 1,
 'good': 1,
 'form': 1}

### 行列への変換

In [ ]:
# DictVectorizerを使用して行列に変換し、datasetに格納する
vectorizer = DictVectorizer(sparse=True)
dataset = vectorizer.fit_transform(word_count_data)

In [ ]:
# datasetの大きさを確認
print("dataset.shape=", dataset.shape)

dataset.shape= (50000, 101249)


In [ ]:
# 各列に対応した単語を取得
feature_names = vectorizer.get_feature_names_out()
print("len(feature_names)=", len(feature_names))
print("最初の20個を表示", feature_names[:20])

len(feature_names)= 101249
最初の20個を表示 ['\x08\x08\x08\x08a' '\x10own' '\\and\\' '`accident' '`action' '`actors'
 '`addicted' '`adelaide' '`adolf' '`adolph' '`adventure' '`afterlife'
 '`agent' '`air' '`alfred' '`alien' '`alive' '`all' '`alone' '`america']


## 4. 機械学習の実施

In [ ]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

目的変数と説明変数を用意します。

In [ ]:
# 目的変数Yの用意
# neg12500 + pos12500 + neg12500 + pos12500 = 50000
Y = np.array(
    [0] * len(train_neg_files) +
    [1] * len(train_pos_files) +
    [0] * len(test_neg_files) +
    [1] * len(test_pos_files)
)

In [ ]:
# 上記のY、および前処理されたdatasetからデータを分割し、
# X_train, Y_train, X_test, Y_testに格納する
#
# 詳細：
#   - dataset の先頭から25000件を 変数 X_train に、残りを変数 X_test に代入
#   - 目的変数 Y の先頭から25000件を 変数 Y_train に、残りを Y_test に代入
X_train = dataset[:25000]
X_test  = dataset[25000:]
Y_train = Y[:25000]
Y_test  = Y[25000:]

In [ ]:
# X_trainとY_trainを、train_test_splitで7:3に分割し、3割のほうを検証データ（X_valid, Y_valid）にする
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X_train, Y_train, test_size=0.3, random_state=RANDOM_STATE, stratify=Y_train
)

In [ ]:
# ロジスティック回帰モデルを作成し、学習して、検証データによる予測を実施する
logreg = LogisticRegression(solver="liblinear", max_iter=1000, random_state=RANDOM_STATE)
logreg.fit(X_train, Y_train)
valid_pred = logreg.predict(X_valid)

# classification_reportを実行し、検証データによるモデルの評価を行なう
print("RANDOM_STATE=",RANDOM_STATE)
print(classification_report(Y_valid, valid_pred, target_names=["neg", "pos"], digits=4))


RANDOM_STATE= 42
              precision    recall  f1-score   support

         neg     0.8795    0.8781    0.8788      3750
         pos     0.8783    0.8797    0.8790      3750

    accuracy                         0.8789      7500
   macro avg     0.8789    0.8789    0.8789      7500
weighted avg     0.8789    0.8789    0.8789      7500



## 5. テストデータによる評価

最後に、テストデータで評価を行ないましょう。

In [ ]:
# テストデータで予測を実施する
test_pred = logreg.predict(X_test)

# classification_reportを実行し、テストデータによるモデルの評価を行なう
print("RANDOM_STATE=",RANDOM_STATE)
print(classification_report(Y_test, test_pred, target_names=["neg", "pos"], digits=4))


RANDOM_STATE= 42
              precision    recall  f1-score   support

         neg     0.8610    0.8690    0.8650     12500
         pos     0.8678    0.8597    0.8637     12500

    accuracy                         0.8644     25000
   macro avg     0.8644    0.8644    0.8644     25000
weighted avg     0.8644    0.8644    0.8644     25000

